# Tarea 7. Experimentacion

Esta tarea consiste en llevar a cabo la experimentación de diversos
clasificadores sobre una variedad de datasets. Se reportarán los resultados y,
posteriormente cuando tengamos la clase de significancia estadística, haremos el
análisis de los clasificadores y si hay un ganador cuál de ellos sería.

Las instrucciones se detallan a continuación:

1. Buscar varios datasets (7, 8, 10, etc.) que cumplan con una condición: deben
   tener alguna característica en común. ¿Qué característica en común? Hay
   varias cosas que pueden satisfacer esta condición, por ejemplo, que traten el
   mismo problema (e.g. diagnósticos médicos aunque sean de diferente
   enfermedad), que tengan muchas más variables que instancias (maldición de la
   dimensionalidad), que todos sean de más de 3 clases, que sus clases estén
   desbalanceadas, entre otras cosas que ustedes puedan identificar. La idea es
   que dichos datasets puedan ser identificados en el mismo contexto.

2. Llevar a cabo la clasificación, utilizando scikit-learn, aplicando los
   clasificadores que hemos visto y algunos otros de su predilección. Para
   realizar la clasificación, deben emplear algún método de validación cruzada
   (k-fold cross-validation, o leave-one-out, dependiendo la cantidad de datos).
   Asimismo, deberán reportar los resultados utilizando el balanced accuracy,
   sensibilidad y especificidad (juntas), o bien el área bajo la curva roc (AUC
   ROC).

3. Posteriormente realizaremos alguna prueba de significancia estadística (cuyo
   tema veremos al regreso de vacaciones), con la finalidad de conocer si
   existen diferencias estadísticamente significativas entre los clasificadores.



# Datos de medidas morfológicas de animales para inferir especie o sexo.

1. [Abalone dataset](https://archive-beta.ics.uci.edu/dataset/1/abalone)

1. [Birds
   bones dataset](https://www.kaggle.com/datasets/zhangjuefei/birds-bones-and-living-habits)

1. [Penguins dataset](https://archive-beta.ics.uci.edu/dataset/690/palmer+penguins-3)

1. [Pokemon_dataset](https://www.kaggle.com/datasets/cristobalmitchell/pokedex)

1. [Sloths dataset](https://www.kaggle.com/datasets/bertiemackie/sloth-species)

In [ ]:
import pandas as pd

abalones = pd.read_csv("../datasets/abalone.csv")
print(abalones.shape)
abalones_ft = abalones.loc[:,['Shell_length', 'Shell_diameter', 'Height', 'Whole_weight',
       'Shucked_weight', 'Viscera_weight', 'Shell_weight',]]
abalones_tg = abalones[['Sex_category']]

birds = pd.read_csv("../datasets/bird.csv")
print(birds.shape)
birds_ft = birds.loc[:,['Humerus_length', 'Humerus_diameter', 'Ulna_length',
       'Ulna_diamater', 'Femur_length', 'Femur_diameter', 'Tibiotarsus_length',
       'Tibiotarsus_diameter', 'Tarsometatarsus_length',
       'Tarsometatarsus_diameter',]]
birds_tg = birds[['Species_group_category']]

penguins = pd.read_csv("../datasets/penguins.csv")
print(penguins.shape)
penguins_ft = penguins.loc[:,['Culmen_Length_mm', 'Culmen_Depth_mm', 'Flipper_Length_', 'Body_Massgr', ]]
penguins_tg = penguins[['Species_category']]

#pokemon = pd.read_csv("../datasets/pokemon.csv")

sloths = pd.read_csv("../datasets/sloth_data.csv")
print(sloths.shape)
sloths_ft = sloths.loc[:,['Claw_length_cm', 'Size_cm', 'Tail_length_cm', 'Weight_kg', ]]
sloths_tg = sloths[['Sub_specie_category']]

datasets = [
    (abalones_ft, abalones_tg),
    (birds_ft, birds_tg),
    (penguins_ft, penguins_tg),
    (sloths_ft, sloths_tg)
]

## Define modelos a utilizar


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

classifiers = []

lr = LogisticRegression()
dtc = DecisionTreeClassifier()
svc = SVC()
knn = KNeighborsClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
#classifiers.append(lr)
classifiers.append(dtc)
classifiers.append(svc)
classifiers.append(knn)
classifiers.append(rfc)
classifiers.append(gbc)


## Validación cruzada (Hold Out)


In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,f1_score
from imblearn.metrics import sensitivity_score, specificity_score, geometric_mean_score

class DatasetSplitter:
    def __init__(self, test_size, random_state=1111):
        self.test_size = test_size
        self.random_state = random_state
    
    def split_datasets(self, datasets, classifiers):
        results = []
        for i, (data, target) in enumerate(datasets):
            X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=self.test_size, random_state=self.random_state)
            classifier = classifiers
            classifier.fit(X_train, y_train.values.ravel())
            y_pred = classifier.predict(X_test)
            k = len(set(y_test))
            if k==2:
              avg = 'binary'
              multi = 'raise'
            else:
              avg = 'macro'
              multi = 'ovr'

            accuracy = accuracy_score(y_test,y_pred)
            sensitivity = sensitivity_score(y_test,y_pred, average=avg)
            specificity = specificity_score(y_test,y_pred, average=avg)
            f1_sc = f1_score(y_test,y_pred, average=avg)
            gmean = geometric_mean_score(y_test,y_pred, average=avg)
            # auc = roc_auc_score(y_test,y_pred,multi_class=multi)
            results.append((accuracy, sensitivity, specificity, f1_sc, gmean))
        results = pd.DataFrame(results, columns=['accuracy', 'sensitivity', 'specificity', 'f1_sc', 'gmean'])
        return results


In [ ]:
splitter = DatasetSplitter(test_size=0.2, random_state=42)

#for model in classifiers:
#    metrics = splitter.split_datasets(datasets, model)
#    print(model.__class__.__name__)
#    print(metrics)

dtc_metrics = splitter.split_datasets(datasets, classifiers[0])
svc_metrics = splitter.split_datasets(datasets, classifiers[1])
knn_metrics = splitter.split_datasets(datasets, classifiers[2])
rfc_metrics = splitter.split_datasets(datasets, classifiers[3])
gbc_metrics = splitter.split_datasets(datasets, classifiers[4])


In [ ]:
my_dict = {
    'Decision Tree classifier': dtc_metrics*100,
    'Support Vector Machine Classifier': svc_metrics*100,
    'K Nearest Neighbour Classifier': knn_metrics*100,
    'Random Forest Classifier': rfc_metrics*100,
    'Gradient Boosting Classifier': gbc_metrics*100
}

In [ ]:
# Print metrics

for key, value in my_dict.items():
       print(key, '\n', value)


## Exactitud (Accuracy)

La exactitud mide la proporción de predicciones correctas del modelo. Es la métrica más básica y se define como el número de predicciones correctas dividido por el número total de predicciones.

In [ ]:
# Plot accuracy metric
for key, value in my_dict.items():
       plt.plot(value['accuracy'], label=key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.ylabel('accuracy')
plt.xlabel('dataset')
plt.title('Metric: accuracy_score')


## Sensitivity score

Es una métrica de evaluación que se utiliza para medir la sensibilidad o la tasa de verdaderos positivos en un problema de clasificación binaria o multiclase, especialmente cuando hay un desequilibrio en la distribución de las clases en los datos de entrenamiento. La sensibilidad se refiere a la capacidad de un modelo para identificar correctamente las muestras positivas, es decir, aquellas que pertenecen a la clase minoritaria o que son relevantes para el problema.

Es una medida de evaluación para abordar el problema de desequilibrio de clases en la clasificación. En comparación con otras métricas de evaluación, como la exactitud, que pueden ser engañosas cuando hay desequilibrio en la distribución de clases, el sensitivity_score proporciona una mejor medida de la capacidad del modelo para detectar correctamente las muestras positivas.

Es especialmente útil en problemas en los que la clase minoritaria es más importante que la clase mayoritaria, como en la detección de enfermedades raras o en la detección de fraudes financieros, entre otros

La sensibilidad macro es adecuada cuando se desea que todas las clases tengan el mismo peso, mientras que la precisión micro es útil cuando se desea dar más peso a las clases con más muestras.



In [ ]:
# Plot sensitivity metric
for key, value in my_dict.items():
       plt.plot(value['sensitivity'], label=key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.ylabel('sensitivity')
plt.xlabel('dataset')
plt.title('Metric: sensitivity_score')

## Especificidad o tasa de verdaderos negativos (Specificity or True Negative Rate)

La especificidad mide la proporción de casos negativos que son correctamente identificados. Se define como el número de verdaderos negativos dividido por la suma de verdaderos negativos y falsos positivos.

In [ ]:
# Plot specificity metric
for key, value in my_dict.items():
       plt.plot(value['specificity'], label=key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.ylabel('specificity')
plt.xlabel('dataset')
plt.title('Metric: specificity_score')

## Valor-F (F-Score)

El valor-F es una media ponderada de la precisión y la sensibilidad. Se define
como 2 veces la multiplicación de la precisión y la sensibilidad dividido por la
suma de la precisión y la sensibilidad.

El valor-F es una métrica que combina tanto la precisión como la sensibilidad en una sola medida. Si nuestro objetivo es encontrar un equilibrio entre la precisión y la sensibilidad, entonces el valor-F es una métrica útil a considerar. Supongamos que para el modelo A, el valor-F es del 0,85, mientras que para el modelo B es del 0,87. En este caso, el modelo B tendría un mejor valor-F, lo que sugiere que es más equilibrado en términos de precisión y sensibilidad.

In [ ]:
# Plot f1 score metric
for key, value in my_dict.items():
       plt.plot(value['f1_sc'], label=key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.ylabel('f1 score')
plt.xlabel('dataset')
plt.title('Metric: F1 score')

## Geometric mean score



In [ ]:
# Plot gmean score metric
for key, value in my_dict.items():
       plt.plot(value['gmean'], label=key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.ylabel('geometric mean score')
plt.xlabel('dataset')
plt.title('Metric: gmean_score')

## Friedman test

La prueba de Friedman es una técnica estadística no paramétrica que se utiliza para comparar el rendimiento de varios clasificadores en diferentes conjuntos de datos. En la práctica, es común evaluar varios modelos de aprendizaje automático en un conjunto de datos específico y comparar sus resultados utilizando diferentes métricas de evaluación.

La prueba de Friedman se utiliza para determinar si existe una diferencia significativa entre los clasificadores en términos de su rendimiento medio. Esta prueba compara los rangos medios de los clasificadores en los diferentes conjuntos de datos para evaluar la hipótesis nula de que todos los clasificadores tienen el mismo rendimiento medio.


In [ ]:
from scipy.stats import friedmanchisquare, rankdata
import numpy as np

acc_score_dict = pd.DataFrame({
    'DTC': dtc_metrics['accuracy'],
    'SVC': svc_metrics['accuracy'],
    'KNN': knn_metrics['accuracy'],
    'RFC': rfc_metrics['accuracy'],
    'GBC': gbc_metrics['accuracy']
})

print(100-acc_score_dict)


In [ ]:
stat, p = friedmanchisquare(*acc_score_dict.values.T)

print(f'\np-value: {p}\n')
print(f'\nstats: {stat}\n')

rank = rankdata(acc_score_dict,axis=1)
mrank = np.average(rank,axis=0)
rank_id = np.argsort(mrank)
rank_sort = np.sort(mrank)

print(f'Tabla de ranking en algoritmos:\n{rank}')
print(f'\nMean-ranks:')
print(mrank)

#print(list(acc_score_dict.keys()))
#print('\nClasificador\t\t\tRanking')
#for idx in rank_id:
#    print(f'{classifiers[idx]} \t\t {mrank[idx]}')
